In [13]:
import torch
from torch.utils import data
import torchvision
import torchvision.transforms as transforms
import numpy as np


In [16]:
# only fit to train-set

transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
print("Size of train-set:", len(trainset))

# as large batch-size as possible! A lower batch-size only makes the variance-estimate more efficient
# to calculate but worsens the estimate!
dataloader = data.DataLoader(trainset, batch_size=4096, shuffle=False, num_workers=4)

pop_mean = []
pop_std = []

for i, batch in enumerate(dataloader):
    # shape (batch_size, 3, height, width)
    numpy_image = batch[0].numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std = np.std(numpy_image, axis=(0,2,3), ddof=1) # sample mean
    
    pop_mean.append(batch_mean)
    pop_std.append(batch_std)

# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
pop_mean = np.array(pop_mean).mean(axis=0)
pop_std = np.array(pop_std).mean(axis=0)


Files already downloaded and verified
Size of train-set: 50000


In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(pop_mean, pop_std)])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
